In [1]:
import yfinance as yf
from datetime import timedelta
import pandas as pd

In [14]:
#Function to Read file of Tickers and Collect Earnings Info and Price Data for PEAD Analysis
def process_ticker(ticker):
    import yfinance as yf
    from datetime import timedelta
    import pandas as pd

    try:
        stock = yf.Ticker(ticker)
        earnings = stock.earnings_dates.reset_index()

        # Clean and rename columns
        earnings.columns = [col.lower().replace(" ", "_") for col in earnings.columns]
        earnings = earnings.rename(columns={
            'earnings_date': 'date',
            'reported_eps': 'reported_eps',
            'eps_estimate': 'eps_estimate',
            'surprise(%)': 'surprise%'
        })

        earnings['date'] = pd.to_datetime(earnings['date'])
        earnings = earnings.dropna()
        #Calculate SUE
        earnings['surprise_raw'] = earnings['reported_eps'] - earnings['eps_estimate']
        std_surprise = earnings['surprise_raw'].std()

        #Avoid divide by zero errors
        earnings['sue'] = earnings['surprise_raw'] / std_surprise if std_surprise > 0 else 0
        earnings['ticker'] = ticker

        # Get price data range
        start_price = earnings['date'].min() - timedelta(days=60)
        end_price = earnings['date'].max() + timedelta(days=100)

        price_data = yf.download(ticker, start=start_price, end=end_price)['Close']
        price_data = price_data.reset_index()
        price_data.columns = ['date', 'adj_close']
        price_data['date'] = pd.to_datetime(price_data['date']).dt.tz_localize(None)

        #Get SPY (S&P 500) data for same period
        spy_data = yf.download('SPY', start=start_price, end=end_price)['Close']
        spy_data = spy_data.reset_index()
        spy_data.columns = ['date', 'spy_close']
        spy_data['date'] = pd.to_datetime(spy_data['date']).dt.tz_localize(None)

        #Merge SPY and Price data
        price_data = pd.merge(price_data, spy_data, on='date', how = 'inner')
        
        results = []

        for _, row in earnings.iterrows():
            event_date = pd.to_datetime(row['date'] + timedelta(days=1)).tz_localize(None)

            for months in [1, 2, 3]:
                end_date = pd.to_datetime(event_date + pd.DateOffset(months=months)).tz_localize(None)

                price_slice = price_data[(price_data['date'] >= event_date) &
                                         (price_data['date'] <= end_date)]

                if price_slice.empty:
                    continue

                start_price = price_slice.iloc[0]['adj_close']
                end_price = price_slice.iloc[-1]['adj_close']
                stock_return = (end_price / start_price) - 1

                start_spy = price_slice.iloc[0]['spy_close']
                end_spy = price_slice.iloc[-1]['spy_close']
                spy_return = (end_spy / start_spy) - 1

                abnormal_return = stock_return - spy_return
                

                results.append({
                    'ticker': ticker,
                    'start_date': event_date.date(),
                    'end_date': end_date.date(),
                    'period_months': months,
                    'cumulative_return': round(stock_return, 4),
                    'sp500_return' : round(spy_return,4),
                    'abnormal_return' : round(abnormal_return, 4),
                    'sue': round(row['sue'], 4),
                    'surprise': round(row['surprise_raw'], 4),
                    'reported_eps': row['reported_eps'],
                    'eps_estimate': row['eps_estimate']
                })

        return results
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return []

#Read File of Tickers
import pandas as pd
tickers_df = pd.read_excel("Meme_Stocks_2023_2024.xlsx", header=None)
tickers = tickers_df[0].dropna().unique().tolist()

#Loop through List of Tickers
all_results = []

for ticker in tickers:
    print(f"Processing {ticker}...")
    all_results.extend(process_ticker(ticker))

#Save result to CSV
final_df = pd.DataFrame(all_results)
final_df.to_csv("PEAD_all_tickers.csv", index=False)
print("Saved PEAD results to PEAD_all_tickers.csv")


Processing AMC...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Processing GME...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Processing BB...


...

Processing DKNG...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Processing HUT...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Saved PEAD results to PEAD_all_tickers.csv
